# 인기검색 종목 가져오기

In [2]:
from request import Request
from bs4 import BeautifulSoup

# Fetch Data
request = Request()
response = request.get(url="https://finance.naver.com")

# get HTML
html = response.text
soup_html = BeautifulSoup(html, "html.parser")

popular_stock_table = soup_html.find(name="div", class_="aside_area aside_popular")
popular_stock = popular_stock_table.find_all(name="tr", class_=["up", "down", "same"])

stock_list = [ stock.find(name="a") for stock in popular_stock ]
print(stock_list)


Request URL - https://finance.naver.com (200)
[<a href="/item/main.naver?code=450080" onclick="clickcr(this, 'boa.list', '450080', '1', event);">에코프로머티</a>, <a href="/item/main.naver?code=417200" onclick="clickcr(this, 'boa.list', '417200', '2', event);">LS머트리얼..</a>, <a href="/item/main.naver?code=086520" onclick="clickcr(this, 'boa.list', '086520', '3', event);">에코프로</a>, <a href="/item/main.naver?code=247540" onclick="clickcr(this, 'boa.list', '247540', '4', event);">에코프로비엠</a>, <a href="/item/main.naver?code=454910" onclick="clickcr(this, 'boa.list', '454910', '5', event);">두산로보틱스</a>]


# 날씨 정보

In [4]:
from request import Request
from beautifulsoup import BeautifulSoupUtil

request = Request()
base_url = "https://www.weather.go.kr/"
response = request.get(url=base_url + "w/index.do#dong/1120069000")

html = response.text
soup_html = BeautifulSoupUtil.parse_html(data=html)

top_10 = soup_html.find(name="div", class_="weblog-top10")
top_10_a = top_10.find_all(name="a")
top_10_list = [ (a.text, base_url + a.get("href")) for a in top_10_a ]

for idx, a in enumerate(top_10_list):
    print(f"Top {idx} : {a}")

Request URL - https://www.weather.go.kr/w/index.do#dong/1120069000 (200)
Top 0 : ('단기예보', 'https://www.weather.go.kr//w/weather/forecast/short-term.do')
Top 1 : ('레이더', 'https://www.weather.go.kr//w/image/radar.do')
Top 2 : ('특보현황', 'https://www.weather.go.kr//w/weather/warning/status.do')
Top 3 : ('산악날씨', 'https://www.weather.go.kr//w/theme/mountain-weather.do')
Top 4 : ('통보문', 'https://www.weather.go.kr//w/weather/warning/list.do')


### 기상특보 발표 기준

In [2]:
from scraping import Scrape

base_url = "https://www.weather.go.kr"
resource_path="/w/community/knowledge/standard.do"

scrape = Scrape(base_url=base_url)
soup_html = scrape.get_html(resource_path=resource_path)
soup_table = soup_html.select_one("body > div.container > section > div > div.cont-wrap > div > div > table")

column_th_list = soup_table.select(selector="thead > tr > th")
super_columns_th = [ column_th_list[i].text for i in range(1, len(column_th_list))]
print(f"Columns : {super_columns_th}")

body_th_list = soup_table.select(selector="tbody > tr > th")
rows = [ th.text for th in body_th_list ]
print(f"Rows : {rows}")

Request URL - https://www.weather.go.kr/w/community/knowledge/standard.do (200)
Columns : ['주의보', '경보']
Rows : ['강풍', '풍랑', '호우', '대설', '건조', '폭풍해일', '한파', '태풍', '황사', '폭염']


### 기상특보 발표 기준 내용

In [53]:
import pandas as pd

soup_content_td = soup_html.select("body > div.container > section > div > div.cont-wrap > div > div > table > tbody > tr > td:not([colspan])")

주의보 = []
경보 = []

for idx, row in enumerate(rows):
    index = idx * 2
    주의보.append(soup_content_td[index].text)
    경보.append(soup_content_td[index+1].text)

dict = {
    "주의보": 주의보,
    "경보": 경보
}
df = pd.DataFrame(data=dict, index=rows)

print(df)

                                                    주의보  \
강풍    육상에서 풍속 50.4km/h(14m/s) 이상 또는 순간풍속 72.0km/h(20...   
풍랑    해상에서 풍속 50.4km/h(14m/s) 이상이 3시간 이상 지속되거나 유의파고가...   
호우    3시간 강우량이 60mm이상 예상되거나  12시간 강우량이 110mm이상 예상될 때...   
대설                                24시간 신적설이 5cm이상 예상될 때   
건조                       실효습도 35%이하가 2일 이상 계속될 것이 예상될 때   
폭풍해일  천문조, 폭풍, 저기압 등의 복합적인 영향으로 해수면이 상승하여 발효기준값 이상이 ...   
한파    10월~4월에 다음 중 하나에 해당하는 경우 ① 아침최저기온이 전날보다 10℃ 이상...   
태풍    태풍으로 인하여 강풍, 풍랑, 호우, 폭풍해일 현상 등이 주의보 기준에 도달할 것으...   
황사    ‘황사주의보’는 ‘미세먼지경보’로 대체(‘17.1.13 시행) * 미세먼지경보에 대...   
폭염    폭염으로 인하여 다음 중 어느 하나에 해당하는 경우\n\t\t\t\t\t\t\t\t...   

                                                     경보  
강풍    육상에서 풍속 75.6km/h(21m/s) 이상 또는 순간풍속 93.6km/h(26...  
풍랑    해상에서 풍속 75.6km/h(21m/s) 이상이 3시간 이상 지속되거나 유의파고가...  
호우    3시간 강우량이 90mm이상 예상되거나 12시간 강우량이 180mm이상 예상될 때\...  
대설    24시간 신적설이 20cm이상 예상될 때. 다만, 산지는 24시간 신적설이 30cm...  
건조                      실효습도 25% 이하가 2일 이상 계속될 것이 예상될 때  
폭풍

# 네이버 금융

In [1]:
from tabnanny import verbose
from scraping import Scrape
import pandas as pd

base_url = "https://finance.naver.com"
resource_path="/sise/dividend_list.naver"

scrape = Scrape(base_url=base_url)
soup = scrape.get_html(resource_path=resource_path)

def get_column_names():
    column_selector = "#contentarea_left > table.type_1.tb_ty > thead > tr"

    super_columns_th = soup.select(column_selector)[0]
    super_cols = [ th.text.strip() for th in super_columns_th if th.text.strip() != "" ]
    sub_columns_th_list = soup.select(column_selector)[1:]

    combine_sub_cols = [
        [ 
            (super_cols[super_cols.__len__() - sub_columns_th_list.__len__() - idx], th.text.strip()) 
            for th in sub_columns_th if th.text.strip() != "" 
        ]
        for idx, sub_columns_th in enumerate(sub_columns_th_list)
    ]

    sub_len_list = [ col.__len__() for col in combine_sub_cols ]
    len_sub_len_list = sub_len_list.__len__()
    flatten_sub_cols = combine_sub_cols[0]
    concat_cols = super_cols[:-len_sub_len_list] + flatten_sub_cols
    return concat_cols

def get_body_dataframe(scrape, columns, page):
    page = 1 if page is None else page
    soup = scrape.get_html(resource_path=resource_path+f"?&page={page}", verbose=0)
    
    body_selector = "#contentarea_left > table.type_1.tb_ty > tbody > tr[class=''],tr[class='tr_even'] > td"
    body_selector = "#contentarea_left > table.type_1.tb_ty > tbody > tr[class=''] > td, #contentarea_left > table.type_1.tb_ty > tbody > tr[class='tr_even'] > td"
    body = soup.select(body_selector)

    column_count = columns.__len__()
    rows = [ row.text for row in body ]
    rows = [ rows[i:i+column_count] for i in range(0, rows.__len__(), column_count) ]
    
    df = pd.DataFrame(data=rows, columns=columns)
    df_name = df["종목명"]
    df = df.iloc[:, 1:].apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors="coerce"))
    df["기준월"] = df["기준월"].astype(object)
    df = pd.concat([df_name, df], axis=1)
    return df

def get_internal_stock_list(max_page):
    column_names = get_column_names()
    df_list = []
    for page in range(1, max_page):
        df = get_body_dataframe(scrape=scrape, columns=column_names, page=page) 
        if df.__len__() == 0:
            print(f"End")
            break
        df_list.append(df)
        print(f"page {page}")
    df = pd.concat(objs=df_list, axis=0)
    return df

df_stock = get_internal_stock_list(max_page=30)
df_stock

Request URL - https://finance.naver.com/sise/dividend_list.naver (200)
parse -> html.parser
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 21
page 22
page 23
page 24
page 25
page 26
page 27
End


,종목명,현재가,기준월,배당금,수익률(%),배당성향(%),ROE(%),PER(배),PBR(배),"(과거 3년 배당금, 1년전)","(과거 3년 배당금, 2년전)","(과거 3년 배당금, 3년전)"
0,한국ANKOR유전,348,22.12,1870,537.36,NaN,NaN,NaN,NaN,180.0,120.0,185.0
1,락앤락,6640,22.12,1953,29.41,-637.03,-2.38,-20.83,0.52,0.0,0.0,0.0
2,한국패러랠,1491,22.03,390,26.16,NaN,NaN,NaN,NaN,90.0,235.0,165.0
3,일성신약,24900,22.12,4000,16.06,28.18,26.59,2.13,0.28,150.0,150.0,150.0
4,리드코프,6170,22.12,800,12.97,64.02,7.47,6.09,0.44,800.0,800.0,150.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9,포스코퓨처엠,365500,22.12,300,0.08,19.64,4.87,117.90,5.64,300.0,285.0,380.0
10,태양금속우,12370,22.12,10,0.08,-34.35,-0.62,-92.29,0.52,10.0,10.0,10.0
11,HPSP,46050,22.12,38,0.08,2.75,53.20,14.81,5.56,0.0,302.0,64.0
12,에코프로,697000,22.12,486,0.07,34.68,2.94,72.63,2.03,410.0,280.0,280.0
